Uso de Gradio para generar la interfaz de prueba de un modelo de clasificación.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import gradio as gr

# Lectura de datos de diabetes
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
columnas = [
    "Embarazos",
    "Glucosa",
    "Presion_sanguinea",
    "Pliegue_cutaneo",
    "Insulina",
    "IMC",
    "DPF",
    "Edad",
    "Resultado",
]
datos = pd.read_csv(url, names=columnas)

# Separación de las características y la variable objetivo
X = datos.drop("Resultado", axis=1)
y = datos["Resultado"]

# División de los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Entrenando el modelo de regresión logística
modelo = LogisticRegression(max_iter=200)
modelo.fit(X_train, y_train)


# Función para hacer predicciones
def predecir_diabetes(embarazos, glucosa, presion, pliegue, insulina, imc, dpf, edad):
    entrada = pd.DataFrame(
        [[embarazos, glucosa, presion, pliegue, insulina, imc, dpf, edad]],
        columns=[
            "Embarazos",
            "Glucosa",
            "Presion_sanguinea",
            "Pliegue_cutaneo",
            "Insulina",
            "IMC",
            "DPF",
            "Edad",
        ],
    )
    prediccion = modelo.predict(entrada)
    probabilidad = modelo.predict_proba(entrada)[0][1]
    resultado = "Positivo" if prediccion[0] else "Negativo"
    return resultado, probabilidad


# Creación de la interfaz con Gradio
inputs = [
    gr.Number(label="Embarazos [0-5]", info="Número de embarazos [0-5]"),
    gr.Number(label="Glucosa [0-199]", minimum=0, maximum=199),
    gr.Number(label="Presión sanguínea"),
    gr.Number(label="Pliegue cutáneo [0-99]"),
    gr.Number(label="Insulina [0-846]"),
    gr.Number(label="IMC [0-67.1]"),
    gr.Number(label="DPF [0.078 - 2.42]"),
    gr.Number(label="Edad [1-81]"),
]
outputs = [
    gr.Label(label="Resultado de predicción"),
    gr.Number(label="Probabilidad (%)"),
]

iface = gr.Interface(fn=predecir_diabetes, inputs=inputs, outputs=outputs)

# Desplegando la interfaz
iface.launch()

In [ ]:
# Deteniendo la demostración
iface.close()